<a href="https://colab.research.google.com/github/sharat-shankar/NLP_J045/blob/master/Classwork/TextBlob_Polarity_and_GloVe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/zfz/twitter_corpus/master/full-corpus.csv")

In [0]:
data = pd.read_json("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz",lines=True)

# **Polarity of Amazon Reviews and Tweets using TextBlob and NLTK**


In [10]:
from textblob import TextBlob
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
nltk.download('vader_lexicon')
sia = SIA()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [0]:
def sentiment(str):
  text1 = TextBlob(str)
  text2 = sia.polarity_scores(str)
  inverse = [(value,key) for key, value in text2.items()]
  result = {"Text Blob":text1.sentiment.polarity,"NLTK":max(inverse)}
  return result

In [0]:
data["Sentiment"] = data.reviewText.apply(sentiment)

In [13]:
data["Sentiment"]

0        {'Text Blob': 0.18000000000000002, 'NLTK': (0....
1        {'Text Blob': 0.17093871975019514, 'NLTK': (0....
2        {'Text Blob': 0.20409090909090905, 'NLTK': (0....
3        {'Text Blob': 0.2142174082747853, 'NLTK': (0.9...
4        {'Text Blob': 0.35, 'NLTK': (0.9455, 'compound')}
                               ...                        
53253    {'Text Blob': 0.19762820512820514, 'NLTK': (0....
53254    {'Text Blob': 0.21251380262249825, 'NLTK': (0....
53255    {'Text Blob': 0.12077922077922078, 'NLTK': (0....
53256    {'Text Blob': 0.46494047619047624, 'NLTK': (0....
53257    {'Text Blob': 0.28119047619047627, 'NLTK': (0....
Name: Sentiment, Length: 53258, dtype: object

In [0]:
df["Sentiments"] = df.TweetText.apply(sentiment)

In [15]:
df["Sentiments"]

0                {'Text Blob': 0.0, 'NLTK': (1.0, 'neu')}
1              {'Text Blob': 0.5, 'NLTK': (0.801, 'neu')}
2       {'Text Blob': 0.39375, 'NLTK': (0.8858, 'compo...
3       {'Text Blob': 0.5416666666666667, 'NLTK': (0.8...
4              {'Text Blob': 0.2, 'NLTK': (0.791, 'neu')}
                              ...                        
5108             {'Text Blob': 0.0, 'NLTK': (1.0, 'neu')}
5109             {'Text Blob': 0.5, 'NLTK': (0.7, 'neu')}
5110         {'Text Blob': -0.25, 'NLTK': (0.833, 'neu')}
5111             {'Text Blob': 0.0, 'NLTK': (1.0, 'neu')}
5112             {'Text Blob': 0.0, 'NLTK': (1.0, 'neu')}
Name: Sentiments, Length: 5113, dtype: object

# **Comparison of Actual Sentiments and Sentiments predicted using Polarity on Tweets**

In [0]:
from sklearn.metrics import confusion_matrix

In [0]:
def final_sentiment(str):
  if str["Text Blob"]>0:
    return 'positive'
  if str["Text Blob"]<0:
    return 'negative'
  if str["Text Blob"]==0:
    return 'neutral'

In [0]:
df["final_sentiment"] = df.Sentiments.apply(final_sentiment)

In [0]:
# Dropping all Tweets with Sentiment Irrelevant since it cannot be classified using Polarity
df = df[df['Sentiment'] != 'irrelevant']

In [20]:
df['Sentiment'] = pd.Categorical(df['Sentiment'])
df['Sentiment'].cat.categories

Index(['negative', 'neutral', 'positive'], dtype='object')

In [21]:
df['final_sentiment'] = pd.Categorical(df['final_sentiment'])
df['final_sentiment'].cat.categories

Index(['negative', 'neutral', 'positive'], dtype='object')

In [22]:
result = confusion_matrix(df["Sentiment"],df["final_sentiment"])
print("Confusion Matrix using TextBlob:")
print(result)

Confusion Matrix using TextBlob:
[[ 234  171  167]
 [ 201 1213  919]
 [  38  124  357]]


In [23]:
from sklearn.metrics import classification_report
print(classification_report(df["Sentiment"],df["final_sentiment"]))

              precision    recall  f1-score   support

    negative       0.49      0.41      0.45       572
     neutral       0.80      0.52      0.63      2333
    positive       0.25      0.69      0.36       519

    accuracy                           0.53      3424
   macro avg       0.52      0.54      0.48      3424
weighted avg       0.67      0.53      0.56      3424



# **Similarity of Reviews and Tweets using GloVe**

In [24]:
!pip install glove_python

     |████████████████████████████████| 266kB 2.7MB/s 
  Created wheel for glove-python: filename=glove_python-0.1.0-cp36-cp36m-linux_x86_64.whl size=700300 sha256=f5de75e5c2b57fc520c71e31783ff3b9e40d1d41c32980f87d32c88753704bc1
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python


In [0]:
from glove import Corpus, Glove

In [26]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
Tweets = df["TweetText"].apply(nltk.word_tokenize)

In [0]:
corpus = Corpus()
corpus.fit(Tweets,window=10)

In [0]:
glove = Glove(no_components=100, learning_rate=0.05)

In [30]:
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)

Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


In [0]:
glove.add_dictionary(corpus.dictionary)

In [35]:
glove.most_similar('siri')

[('ios5', 0.9929521752923832),
 ('iphone4s', 0.9883325135859712),
 ('TWITTER', 0.9700626210539401),
 ('iphone4S', 0.9699050519069947)]